In [1]:
family_group = "4.Pierinae"


def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    if "2.Final_output" in list_of_folders:
        subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [2]:
import os
from Bio import SeqIO
import subprocess



output_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/{family_group}"
make_output_folder(output_location)
genome_location = "/mnt/f/Genomes_2023-12-26"
list_of_species = os.listdir(f"{output_location}/1.Blast_result")
if 'desktop.ini' in list_of_species:
    list_of_species.remove("desktop.ini")
# print(list_of_species)
# list_of_species = ["Heliconius_charithonia"]
# list_of_species = ["Vanessa_tameamea"]
error_exons = ''
for species in list_of_species:
    print(species)
    genome_file = get_genome_file(genome_location,species)
    print(genome_file)
    previous_coordinate_start = ''
    previous_scaffold = ''
    with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
        coordinate_file_list = coordinate_final_file.readlines()
    
    for i in range(1,len(coordinate_file_list)):
        
        coordinate_line_split = coordinate_file_list[i].split(",")
        
            
#         print(coodinate_line_split[5])
        if coordinate_line_split[5] == "N":
            if previous_coordinate_start == '':
                previous_coordinate_start = coordinate_line_split[2]
                previous_scaffold = coordinate_line_split[1]
            complement = coordinate_line_split[4]
            start,stop = coordinate_line_split[2],coordinate_line_split[3]
            print(start,stop)
            query_name = coordinate_line_split[6]
            scaffold = coordinate_line_split[1]
            exon = f"Exon_{query_name.split('_')[-1]}"
            left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
#             print(complement)
            if complement == "0":
                if start < previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            elif complement == "1":
                if start > previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            else:
                print("ERROR")
            
        elif coordinate_line_split[5] == "Y":
            error_exons += coordinate_file_list[i]
        
        gene_sequence = get_gene_sequence(genome_location, 
                                          species, 
                                          genome_file, 
                                          scaffold,
                                          start,
                                          stop,
                                          complement,
                                          output_location)
        
       
        current_exon = query_name.split('_')[-1]
        if current_exon == "1":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()
            print(translated)
            if "*" in translated:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
        
                write_exon_output(output_location,species,exon,exon_output)
        elif current_exon == "24":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()

            if "*" in translated[:-1]:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                print(translated)
            else:
                
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
                write_exon_output(output_location,species,exon,exon_output)
#         assert False
        else:
            exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
            print(exon_output)

            write_exon_output(output_location,species,exon,exon_output)
        
#         assert False
        previous_coordinate_start = start
        previous_scaffold = scaffold
    error_exons += "\n\n"
#     print(coordinate_file_list)
#     break
coordinate_file = ''
print(error_exons)
with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
    error_out_file.write(error_exons)

Anthocharis_cardamines
GCA_905404175.1_ilAntCard3.1_genomic.fna
4965981 4966057
Getting Gene


/home/saurav/.local/lib/python3.8/site-packages/Bio/Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


MDNLDDNAKVSDSAYSNSCSNSQAR
>Anthocharis_cardamines_Exon_1_FR989951.1_4965981_4966057_left_0_right_2
ATGGACAACTTAGATGACAATGCCAAGGTCTCAGATTCAGCATATTCTAACAGCTGTAGCAATAGCCAAGCTAGAAg
4965134 4965217
Getting Gene
>Anthocharis_cardamines_Exon_2_FR989951.1_4965134_4965217_left_1_right_2
cCGTAGTTCACAATCGACGCATTCAGGAAGCCACTCCAGCGGAAGCAGTGGATACGGCGGAAAGCCCTCCACTTCTGgatatag
4962683 4962831
Getting Gene
>Anthocharis_cardamines_Exon_3_FR989951.1_4962683_4962831_left_1_right_1
cAACAAAGTCTGTCAACCCCCTGAAATAAGatgtaaaataaagaaaaagaagaagcaACCAGACGAGAGTGAAACGATTCAAGAAGAAAAGCCGAGTGCTCCCGAACCTGAACCAGTAGTGGAGCCGGCGAAAGAAGTGTGCGTTGAAG
4961428 4961519
Getting Gene
>Anthocharis_cardamines_Exon_4_FR989951.1_4961428_4961519_left_2_right_0
CTGTTAGTTCACCTATCGTACAGCAAGCGGAGTCAGCACAACTGGAAACATTAGAGATATCGACTACAGATAGAAGTAGCGTCAAGGATAAG
4960955 4961014
Getting Gene
>Anthocharis_cardamines_Exon_5_FR989951.1_4960955_4961014_left_0_right_0
ACTAGCACGTGCAACTTACCAGTCGCACGCACACCGAATCGGACTTTAGCTAAAAGTTGT
4960110 4960241
Getting Ge

3368021 3368170
Getting Gene
>Aporia_crataegi_Exon_12_OU538732.1_3368021_3368170_left_0_right_0
ATATTGACGAAGCCGGCGGAAGCAGCAAAGCAACGAATGACGAAACGCTGTCAAGATTTGGCATCGTTCATGGACGGTCTAATTGAAGAGGCACCAAAGACTGATGACGATATATGGCTAGATATTAAGCAGGCTGATAGCGTCTATTAT
3367345 3367478
Getting Gene
>Aporia_crataegi_Exon_13_OU538732.1_3367345_3367478_left_0_right_2
GACTTTACGGTGCAAGAGCGAGATTCTGTACTGCTAGGAGGAATTTCCCCTCACCACGATTCCAGCGACAGTAAATCTAGTACTGAAACACAAATGAGTTACAACCAGCTAAATTACAGCGAGACTCTGCAACG
3366601 3366672
Getting Gene
>Aporia_crataegi_Exon_14_OU538732.1_3366601_3366672_left_1_right_2
GTACTTCGATAGCCACTTAATATATTCTGGGCGAAACAAACCAACGTGCAACAGCAAATTGAACTGTACCgg
3366255 3366395
Getting Gene
>Aporia_crataegi_Exon_15_OU538732.1_3366255_3366395_left_1_right_2
GCGATGCATGTCGCCAATGGACCAATATTATGGCGACTCAGGTGATATGACCAGCTCGTGCGATTCAAGTGCAGCCATTTTTGCAAGTTGTAGTCCTAGAAACTACTCCTCTGTCAGATTGACAGAATCTGCATTGAATaa
3365841 3366018
Getting Gene
>Aporia_crataegi_Exon_16_OU538732.1_3365841_3366018_left_1_right_0
ACATAACACAATAATGAAG

>Leptophobia_aripa_Exon_23_OX637275.1_6791346_6791495_left_0_right_0
AGTACAACTACCACAGAGACGGAACCGAAGCCGCAGCAAACAGCAAGCGAAGATACAATTGACAAAACCGATGAGTCGAGCTATTCATCGTTCTATTCCAATTTCTTTAAGAGTGAATCGCGGAGCGCAGAAGAGAGTGATGTTAAAAAG
6791045 6791128
Getting Gene
>Leptophobia_aripa_Exon_24_OX637275.1_6791045_6791128_left_0_right_0
CTATCGGGCGCATCCAGTTCAATAAGTGCAAAGGATTCTCTTAAGGAGCTTAGACGTAAGATGGAACCGCCGTGGATGGAACAT
6790462 6790563
Getting Gene
>Leptophobia_aripa_Exon_25_OX637275.1_6790462_6790563_left_0_right_0
GTTTGCGTGACATCCGAACTTATATACCAATATCAAGTGTTACCGAAGAATATTGAAGATGTATTGTCACGTGATCAGGAGCGATTGAAAACATTGGAACAG
6789023 6789124
Getting Gene
>Leptophobia_aripa_Exon_26_OX637275.1_6789023_6789124_left_0_right_0
GTGCGCCGACGTAAACGTGAATACAGCAAGCTGGTTATGATATACGAAGAGGACGCACCACTCCCGCCACCAGAAGAGCCAGCGAGTACTTCCACATCATAA
Pieris_brassicae
GCF_905147105.1_ilPieBrab1.1_genomic.fna
4554119 4554204
Getting Gene
MGNIDDAENNAKISDSAYTNSCSNSQAR
>Pieris_brassicae_Exon_1_NC_059680.1_4554119_4554204_left_0_right_2
ATGGGTAATAT

>Pieris_mannii_Exon_9_CM054800.1_4444282_4444510_left_2_right_2
TCCCTAATGAAATTATATCGAAACCTATCCCGTTCGTTATAAGGCACGCGGCAAATGGTGATATAACGTACATAGACCCTGAATCTGTGCCGTACTTGGGATATTCACCTcaagatatattaaatatggaCGCACTTCATCTATACCATCCACACGACCTTGCATATTTAAGACAAGTGTACgaaacaattgttaaaaaaggTGGTGTGCTGAGATCAAAACCATATAG
4445047 4445158
Getting Gene
>Pieris_mannii_Exon_10_CM054800.1_4445047_4445158_left_1_right_0
AATGATGACTCAAAACGGTGACTATGTTAAACTCCAAACCGAATGGTCGTCATTTATAAACCCGTGGTCTAGAAAATTGGAATTCGTTATTGGAAAACATATGCTGATTGAG
4445405 4445518
Getting Gene
>Pieris_mannii_Exon_11_CM054800.1_4445405_4445518_left_0_right_0
GGTCCAACTAATCCTGATGTATTTCAATGTGCcgaaaaatctataaaattttctGAAGATGAGAAGAAAATAGCACAAGCCATGAGAGAAAGCATAATTAGGGTCATGAATAAG
4448088 4448228
Getting Gene
>Pieris_mannii_Exon_12_CM054800.1_4448088_4448228_left_0_right_0
ATATTAACAAAGCCAGCGGAAGCGGCGAAGCAACAAATGACCAAACGTTGCCAAGATTTAGCCTCGTTCATGGAGAGTCTAATGGACGAAGCCCCAAAGCACGATGAAGATATCTGGGTTGATGTTAAGGAGCCTGATTAC
4448590 4448723
Getting Gene
>Pieris_mannii_Exon_13_CM

>Pieris_melete_Exon_19_CAVNZK010000320.1_2751896_2751997_left_2_right_1
gTATGGTTCCCGTTTGCTACGTGCCTGCAACGCAACAGCAAATGACAGGGGACGGAACGACAAACAAACCTAATCAAAACACCGTCAATCAGGCATATGCTA
Getting Gene
>Pieris_melete_Exon_19_CAVNZK010000320.1_2751896_2751997_left_2_right_1
gTATGGTTCCCGTTTGCTACGTGCCTGCAACGCAACAGCAAATGACAGGGGACGGAACGACAAACAAACCTAATCAAAACACCGTCAATCAGGCATATGCTA
2751544 2751773
Getting Gene
>Pieris_melete_Exon_21_CAVNZK010000320.1_2751544_2751773_left_2_right_0
tGCCTTGTGTAATGTATGGTCAGCCGATTTATAGTACCCCGTTAATGTATTCGGCGGTAAACCCACAAATGCAACAGATGCAGAACATGCAACAGATTCAGCATATGCAACAAATGCAGCACATGCATCAAATGCAACAAATGCAACAGATGCAGCAAGATTATATGGCACAAAACTTTGATATAGCGACAATGCATTCCCTTAGACTGCCAAGCACCACTTATCAGGAG
2750934 2751071
Getting Gene
>Pieris_melete_Exon_22_CAVNZK010000320.1_2750934_2751071_left_0_right_0
GCTTGTAAACTCACTTTTCCAACGGAGAGCAAGTCGATCACTACAGCAGTATGGAAAGAGCAACGGGAAGTACCACTGAATCAATCAAAGAAAGAAGGAAGTGGTGCCAGTCGTTCATCAGCCGCTGTTTCCCTGCTG
2750154 2750306
Getting Gene
>Pieris_melete_Exon_23_CAVNZK0100003

>Pieris_rapae_Exon_4_NC_059534.1_3900800_3900900_left_2_right_0
ATACTAGCTCACCTGCCGTGCCGCAAAAAGACGTATTGCAAGTTGAAATTAGTGATACACCGACTACAGAGAgaaataattcaaaggaaaaaaaagaaaag
3901101 3901148
Getting Gene
>Pieris_rapae_Exon_5_NC_059534.1_3901101_3901148_left_0_right_0
aCAAGTACGTGCAATATCCCGATACAAACaccaataaaaacattgttg
3901492 3901623
Getting Gene
>Pieris_rapae_Exon_6_NC_059534.1_3901492_3901623_left_0_right_0
GATGGCTTTTCGTGTGTGATATCTATGCAAGACGGAGTGGTTATGTACACAACAAATTCCCTTACAAACACATTGGGTTTCCCAAGTGATATGTGGATCGGACGGTCCTTCATTGACTTCGTACATCCTAGG
3903277 3903352
Getting Gene
>Pieris_rapae_Exon_7_NC_059534.1_3903277_3903352_left_0_right_1
GACCGGAACACGTTCGCATCACAAATAACGAACAGTGTTGCTATGCCAGAAATTTTAAATGGTTCTCATAACaaaa
3903671 3903889
Getting Gene
>Pieris_rapae_Exon_8_NC_059534.1_3903671_3903889_left_2_right_1
CAAGCATAAAACTTTCTTTTTGTAAAGGTAGCACACTTTCCACTATGGTGTGTCGTATTCGCAAATACCGTGGACTGGGAGGTGGATACGGTGTAAAGAATAAAATGGTTTCATTTTTGCCATTCTTACTTAAACTATCCTTTAGAAATATAACTGATGAGGAAGGCGaggttatatatttacttctGCAGGCAACGCCGTTT